Import libraries

In [8]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Create Dataframe for Toronto Data - The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [11]:
df = pd.read_csv("ToNeigh.csv", names=['PostalCode', 'Borough', 'Neighborhood'])


FileNotFoundError: [Errno 2] File b'ToNeigh.csv' does not exist: b'ToNeigh.csv'

Remove rows with Neighborhood = Not assigned, Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned. 

In [ ]:
df1 = df[df.Borough!='Not assigned']
df1.reset_index(inplace=True)

If More than one neighborhood can exist in one postal code area combined into one row with the neighborhoods separated with a comma 
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

In [ ]:
tempPC = 'MMM'
tempNH = 'NNN'
for x in range(df1.shape[0]):
    if df1.Neighborhood[df1.index==x].values[0] == 'Not assigned':
        df1.at[x,'Neighborhood'] = df1.Borough[df1.index==x].values[0]
        
    if df1.PostalCode[df1.index==x].values[0] == tempPC:  
        df1.at[x,'Neighborhood'] = tempNH + ', ' + df1.Neighborhood[df1.index==x].values[0]
        df1.at[x-1,'Neighborhood'] = 'Delete'
    tempPC =  df1.PostalCode[df1.index==x].values[0]
    tempNH =  df1.Neighborhood[df1.index==x].values[0]
df2 = df1[df1.Neighborhood!='Delete']    

Reindex


In [ ]:
df2.reset_index(inplace=True)
del df2['index']
del df2['level_0']
df2.head()

Use the .shape method to print the number of rows of your dataframe.

In [ ]:
df2.shape